In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/5.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/3.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112045232,NM5M,0
1,1112486703,NM5M,0
2,1112463030,NM5M,0
3,1112017891,NM5M,0
4,1112445183,5awk,0
5,1111353195,5awk,0
6,1112119639,5awk,0
7,1112466739,5awk,0
8,1112107934,5awk,0
9,1112261241,5awk,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16455043


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
7883970,1112429440,4rPZzZ5,0,FEM,26.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
4509375,1112286522,GNJv4RA,0,MASC,25.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
15455405,1112341765,ZOejK8,0,MASC,32.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
9969388,1112229354,bOj1d2B,1,MASC,22.0,-,-,-,-,-,...,False,True,False,False,False,True,False,True,False,NaN
15292323,1111765305,6r2kON3,0,FEM,31.0,-,-,Graduado,-,-,...,False,True,False,False,False,False,False,False,False,NaN
14422348,1112325437,96ZOA3Z,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
9542442,1112452103,64bLOj,1,MASC,28.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.001818
13576708,1111686486,0zP08Dr,0,MASC,22.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
15580496,1112355121,ow20YVe,0,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,NaN
11732038,1112411633,1ZdLVE,0,MASC,57.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
7141768,1112316657,8QlXr3,0,FEM,32.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,0.000000
9754172,1112426761,vVjbGAX,1,FEM,55.0,-,-,-,Graduado,Graduado,...,False,False,True,False,False,False,False,False,False,0.111111
12928759,1112367899,YjAe8NV,0,FEM,20.0,-,-,Graduado,-,-,...,False,False,True,False,False,False,False,True,False,0.000000
6395965,1112339930,VNJmpvz,1,MASC,42.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
8863263,1112432391,8MP8E6z,0,FEM,43.0,-,-,Graduado,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.000000
6942640,1112283554,kP6o5rV,1,FEM,25.0,-,-,Graduado,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000
12364197,1112444553,536R1k,0,FEM,27.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
7568474,1112264061,qekpkAL,0,FEM,35.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.000000
10614694,1112396117,8jPpBz,0,FEM,54.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
14721816,1112288883,RzrQbBO,1,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15040283 Test:  791594


In [16]:
gc.collect()

156

In [17]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

In [18]:
score=c.score(x_test,y_test)*100
print(score)

77.48176464197556


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV14.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV14.pkl']

In [20]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
2884,74837,1112463897,EPdBv0,FEM,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.009259
84511,45336,1112428183,88joxR,FEM,32.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
89322,49774,1112438527,5d1oYZ,MASC,38.0,-,-,-,Graduado,-,...,False,False,False,False,False,False,False,False,False,0.000000
92179,91157,1112470541,1lGv8L,FEM,45.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.000000
96885,96564,1112494684,YRNqdr,MASC,41.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,0.000000
37686,71165,1112462019,96M1Dxw,MASC,29.0,-,-,-,-,Graduado,...,False,True,True,False,False,False,False,False,False,0.020833
80344,64404,1112458199,kPjkLaV,FEM,42.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.100000
51864,3919,1111417142,e0YwEE,MASC,43.0,-,-,Graduado,-,-,...,False,True,False,False,False,False,False,False,False,0.000000
21314,82192,1112466195,mzb5qOQ,FEM,26.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.019608
7685,89181,1112467020,ekojk52,FEM,22.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.015152


In [21]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.76953096, 0.23046904],
       [0.80345631, 0.19654369],
       [0.7779683 , 0.2220317 ],
       ...,
       [0.94066048, 0.05933952],
       [0.87010911, 0.12989089],
       [0.96729075, 0.03270925]])

In [23]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [24]:
prediccion['sepostulo']=y_final2

In [25]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.230469
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.196544
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.222032
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.086806
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.032458


In [26]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.230469
1,1,0.196544
2,2,0.222032
3,3,0.086806
4,4,0.032458


In [27]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion29.csv",index=False)

In [28]:
prediccion.sepostulo.mean()

0.5210018234370081